<a href="https://colab.research.google.com/github/lucasfelipecdm/tech-challenge-fase-3/blob/main/preprocessing_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tech Challenge 3
### Fine Tuning com Foundation Model GPT-3.5-turbo da OPENAI

Para o projeto de Tech Challenge da fase 3 do curso de AI para desenvolvedores da FIAP, decidimos realizar o fine tuning, processo de ajuste fino para refinamento de modelos pré-treinados, utilizando o modelo GPT 3.5 Turbo da OPENAI, o modelo é pago, por isso para evitar gastos altos decidimos realizar alguns ajustes no nosso dataset.

O dataset utilizado foi o sugerido [AmazonTitle1MM](https://drive.google.com/file/d/12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK/view) - realizamos seu download e a extração dos dados do json: **"trn.json"**. Decidimos utilizar somente as 100 primeiras linhas e criamos um **"trn_smaller.json"** salvando-o em uma pasta dentro do nosso drive.

Vamos para o processo de pré-processamento dos dados e do Fine Tuning:

#### Instalação das bibliotecas externas necessárias

- Openai para acesso ao foundation model do GPT e a sua API;
- Python-dotenv para carregamento do arquivo .env contendo nossa chave de acesso a Openai

In [ ]:
!pip install openai
!pip install python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


#### Acesso ao nosso drive do Google

Aqui realizamos o acesso ao nosso drive para buscar e salvar dados processados e também carregar nossas variáveis de ambiente

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Pré-processamentos e formatação de dados para Fine Tuning

Realizamos aqui a preparação dos dados e também a formatação e organização dos dados para que estejam da forma sugerida do modelo que selecionamos, neste caso **"gpt-3.5-turbo"** para esse modelo temos que seguir o seguinte formado sugerido na documentação: [Fine tuning - Example Format](https://platform.openai.com/docs/guides/fine-tuning/example-format)



```
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "Oh, just some guy named William Shakespeare. Ever heard of him?"}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters."}]}
```

Além de alterar os dados para se ajustarem ao formato recomendado, decidimos retirar todas as linhas onde o conteúdo é vázio, isso acontece em quase 50% do dataset que selecionamos.

In [ ]:
import json

def prepare_data_for_finetuning(dataset_path, output_path):
    with open(dataset_path, 'r') as file:
        data = [json.loads(line) for line in file]

    prepared_data = []

    for item in data:
        if 'title' in item and item.get('content', '').strip():
            system_content = "You are a helpful assistant that provides detailed product information based on titles and descriptions."
            user_content = f"What's the description of {item['title']}?"
            assistant_content = f"Here's what I found about the product {item['title']}: {item['content']}"

            message_format = {
                "messages": [
                    {"role": "system", "content": system_content},
                    {"role": "user", "content": user_content},
                    {"role": "assistant", "content": assistant_content}
                ]
            }

            prepared_data.append(message_format)

    with open(output_path, 'w') as f:
        for entry in prepared_data:
            f.write(json.dumps(entry) + '\n')

prepare_data_for_finetuning("/content/drive/MyDrive/Colab Notebooks/tech-challenge-3-content/trn_smaller.json", "/content/drive/MyDrive/Colab Notebooks/tech-challenge-3-content/trn_smaller_pre_processed.jsonl")

#### Carregamento de Variáveis de Ambiente e Acesso a OpenAI

Aqui utilizamos as bibliotecas pré-instaladas para carregar o arquivo .env com nossa chave para acesso a API da OpenAI e também criamos um client para acesso ao upload de arquivos e criação do fine tuning model.

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv('/content/drive/MyDrive/Colab Notebooks/tech-challenge-3-content/.env')
client = OpenAI(api_key=os.environ.get('OPENAI_KEY'))

#### Upload do arquivo pré-processado

Realizamos aqui o upload do arquivo com os dados pré-processado anteriorment e salvo em nosso drive para o nosso storage na OpenAI, dessa forma o processo de Fine Tuning tem acesso ao nossos dados de treinamento.

In [ ]:
upload_response = client.files.create(
  file=open("/content/drive/MyDrive/Colab Notebooks/tech-challenge-3-content/trn_smaller_pre_processed.jsonl", "rb"),
  purpose='fine-tune'
)
file_id = upload_response.id
print(f"Pre-processed data file successfully uploaded! [File ID: {file_id}]")

Pre-processed data file successfully uploaded! [File ID: file-G086iCezJDKN5OJb73kyKEwt]


#### Criação do Fine Tuning

Aqui realizamos a chamada para inicio de Fine Tuning, passando como parametros o modelo que desejamos utilizar e também o ID do arquivo que deve ser utilizado para treimento do modelo e fine tuning.

In [ ]:
fine_tune_response = client.fine_tuning.jobs.create(
  training_file=file_id,
  model="gpt-3.5-turbo"
)
fine_tune_response

FineTuningJob(id='ftjob-TThnFDo13WG0NmuZbdH5xIil', created_at=1727292436, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-hPjeEVgMmlY5VHiQLpguka6k', result_files=[], seed=767339603, status='validating_files', trained_tokens=None, training_file='file-G086iCezJDKN5OJb73kyKEwt', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

#### Verificar o status do Fine Tuning

Aqui podemos verificar o status do Fine Tuning utilizando o ID do processo que recebemos na etapa anterior, também é possivel acompanhar caso acontece alguma falha ou erro durante o fine tuning.

In [ ]:
retrieve_response = client.fine_tuning.jobs.retrieve(fine_tune_response.id)
retrieve_response

FineTuningJob(id='ftjob-TThnFDo13WG0NmuZbdH5xIil', created_at=1727292436, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::ABS2yQYP', finished_at=1727292783, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-hPjeEVgMmlY5VHiQLpguka6k', result_files=['file-lrb1Oqv0LQSWYyirEh2DyOqa'], seed=767339603, status='succeeded', trained_tokens=27318, training_file='file-G086iCezJDKN5OJb73kyKEwt', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

#### Utilizando o modelo treinado

Aqui fazemos uma simples condição que verifica se o processo de Fine Tuning foi finalizado e caso sim, adicionamos o ID a uma váriavel para utilização durante o teste.

In [ ]:
if retrieve_response.fine_tuned_model != None:
    fine_tuned_model = retrieve_response.fine_tuned_model
fine_tuned_model

'ft:gpt-3.5-turbo-0125:personal::ABS2yQYP'

#### Teste da pergunta do usuário a um modelo sem Fine Tuning

Aqui utilizamos uma chamada normal ao completions da OpenAI utilizando um modelo normal sem fine tuning 'gtp-3.5-turbo', nas mensagens passamos parametros como qual a role de quem está fazendo a pergunta, o número máximo de tokens para uma resposta mais completa ou não e também a temperatura dizendo o quanto queremos de liberdade para o modelo, para que ele seja criativo ou mais específico:

In [ ]:
new_prompt = "What's the description of Girls Ballet Tutu Neon Blue?"

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[{'role': 'user', 'content': new_prompt}],
  max_tokens=200,
  temperature=0.2
)
print(completion.choices[0].message.content)

The Girls Ballet Tutu in Neon Blue is a vibrant and eye-catching tutu skirt perfect for young dancers. The tutu features multiple layers of tulle in a bright neon blue color, creating a fun and playful look. The elastic waistband ensures a comfortable and secure fit, making it easy to move and dance in. This tutu is perfect for ballet classes, recitals, dress-up play, or any other special occasion where your little dancer wants to stand out and shine.


#### Teste da pergunta do usuário a um modelo com Fine Tuning

Aqui utilizamos uma chamada ao completions da OpenAI utilizando o modelo com Fine Tuning, utilizamos os mesmos parametros para message, max tokens e temperature:

In [ ]:
new_prompt = "What's the description of Girls Ballet Tutu Neon Blue?"

completion = client.chat.completions.create(
  model=fine_tuned_model,
  messages=[{'role': 'user', 'content': new_prompt}],
  max_tokens=200,
  temperature=0.2
)
print(completion.choices[0].message.content)

Here's a summary of the product Girls Ballet Tutu Neon Blue: This is a beautiful neon blue tutu. The skirt is made of a soft tulle and the waistband is made of a soft satin. This tutu is perfect for a dance recital or to wear just for fun.


#### Comparação das respostas de antes e depois do Fine Tuning:

Descrição no dataset da AmazonTitles1MM:
> Dance tutu for girls ages 2-8 years. Perfect for dance practice, recitals and performances, costumes or just for fun!

Descrição do modelo **SEM** Fine Tuning:
> The Girls Ballet Tutu in Neon Blue is a vibrant and eye-catching tutu skirt perfect for young dancers. The tutu features multiple layers of tulle in a bright neon blue color, creating a fun and playful look. The elastic waistband ensures a comfortable and secure fit, making it easy to move and dance in. This tutu is perfect for ballet classes, recitals, dress-up play, or any other special occasion where your little dancer wants to stand out and shine.

Descrição do modelo **COM** Fine Tuning:
> Here's a summary of the product Girls Ballet Tutu Neon Blue: This is a beautiful neon blue tutu. The skirt is made of a soft tulle and the waistband is made of a soft satin. This tutu is perfect for a dance recital or to wear just for fun.



